<a href="https://colab.research.google.com/github/yhpark1962/baseball/blob/main/news_best1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 필요한 라이브러리 설치 및 가져오기
!pip install requests
!pip install pandas
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

# 검색어 입력
query = input("keyword to search: ")

# Google 뉴스 검색 URL (start 파라미터로 페이지 변경)
base_url = f"https://www.google.com/search?q={query}&tbm=nws"

# HTTP 요청 헤더 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7"
}

# 뉴스 데이터 추출
titles = []
contents = []
links = []
dates = []

# 원하는 뉴스 갯수 설정
max_news = 40
pages_to_fetch = (max_news // 10) + (1 if max_news % 10 > 0 else 0)  # 필요한 페이지 수

def fetch_news_page(page):
    """개별 페이지에서 뉴스 데이터를 크롤링"""
    url = f"{base_url}&start={page * 10}"
    response = requests.get(url, headers=headers)
    soup = bs(response.text, "html.parser")

    page_titles = []
    page_contents = []
    page_links = []
    page_dates = []

    news_items = soup.select(".SoaBEf")
    for item in news_items:
        title = item.select_one(".nDgy9d").text if item.select_one(".nDgy9d") else "제목 없음"
        content = item.select_one(".Y3v8qd").text if item.select_one(".Y3v8qd") else "본문 없음"
        link = item.select_one("a").get("href") if item.select_one("a") else "링크 없음"

        # 날짜 추출
        date_text = item.select_one(".WG9SHc")
        if date_text:
            date_text = date_text.text.strip()
        else:
            date_text = None

        if date_text and "오늘" in date_text:
            news_date = datetime.today().date()
        else:
            if date_text:
                try:
                    news_date = datetime.strptime(date_text, "%Y.%m.%d").date()
                except ValueError:
                    try:
                        news_date = datetime.strptime(date_text, "%Y-%m-%d").date()
                    except ValueError:
                        try:
                            news_date = datetime.strptime(date_text, "%Y/%m/%d").date()
                        except ValueError:
                            news_date = None
            else:
                news_date = None

        page_titles.append(title)
        page_contents.append(content)
        page_links.append(link)
        page_dates.append(news_date)

    return page_titles, page_contents, page_links, page_dates

# 병렬로 페이지 요청
with ThreadPoolExecutor() as executor:
    results = list(executor.map(fetch_news_page, range(pages_to_fetch)))

# 결과 병합
for page_titles, page_contents, page_links, page_dates in results:
    titles.extend(page_titles)
    contents.extend(page_contents)
    links.extend(page_links)
    dates.extend(page_dates)

# 데이터프레임 생성
news_df = pd.DataFrame({
    "Titles": titles[:max_news],  # 최대 뉴스 수 제한
    "Date": dates[:max_news],
    "Contents": contents[:max_news],
    "Links": links[:max_news]
})

# 최신 뉴스가 먼저 오도록 날짜 기준으로 정렬
news_df = news_df.sort_values(by="Date", ascending=False)

# Colab 환경에서 파일 저장 경로 설정
output_path = "/content/news_crawling.csv"

# CSV로 저장
news_df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"news data saved: {output_path}")

# 데이터 미리보기
news_df.head()


keyword to search: 인공지능
news data saved: /content/news_crawling.csv


,Titles,Date,Contents,Links
0,"젠슨 황, 인공지능 추론 시대 여는 ‘블랙웰 울트라’ 공개",None,본문 없음,https://www.hani.co.kr/arti/international/amer...
1,옥천군 '인공지능 행정' 도입 속도…도내 첫 자치법규 제정,None,본문 없음,https://news.nate.com/view/20250321n08622
2,"엔비디아 AI 가속기 칩 ‘블랙웰’, 발열과의 전쟁",None,본문 없음,https://www.chosun.com/economy/tech_it/2025/03...
3,“AI는 위협·기회 공존…인간에 대한 신뢰 바탕 개발돼야”,None,본문 없음,http://www.koreatimes.com/article/20250320/155...
4,"참외도 인공지능, AI로 재배…1조 원 시대 연다",None,본문 없음,https://news.nate.com/view/20250321n09125
